# DCP Testing

In [16]:
from __future__ import print_function
import os
import gc
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import sys
sys.path.append("dcp-master")
# from data import ModelNet40
from model import DCP
from util import transform_point_cloud, npmat2euler, quat2mat
import numpy as np
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from tqdm import tqdm


# Part of the code is referred from: https://github.com/floodsung/LearningToCompare_FSL

class IOStream:
    def __init__(self, path):
        self.f = open(path, 'a')

    def cprint(self, text):
        print(text)
        self.f.write(text + '\n')
        self.f.flush()

    def close(self):
        self.f.close()


def _init_(args):
    if not os.path.exists('checkpoints'):
        os.makedirs('checkpoints')
    if not os.path.exists('checkpoints/' + args.exp_name):
        os.makedirs('checkpoints/' + args.exp_name)
    if not os.path.exists('checkpoints/' + args.exp_name + '/' + 'models'):
        os.makedirs('checkpoints/' + args.exp_name + '/' + 'models')
    os.system('cp main.py checkpoints' + '/' + args.exp_name + '/' + 'main.py.backup')
    os.system('cp model.py checkpoints' + '/' + args.exp_name + '/' + 'model.py.backup')
    os.system('cp data.py checkpoints' + '/' + args.exp_name + '/' + 'data.py.backup')

## Train

In [7]:
def train_one_epoch(args, net, train_loader, opt):
    net.train()

    mse_ab = 0
    mae_ab = 0
    mse_ba = 0
    mae_ba = 0

    total_loss = 0
    total_cycle_loss = 0
    num_examples = 0
    rotations_ab = []
    translations_ab = []
    rotations_ab_pred = []
    translations_ab_pred = []

    rotations_ba = []
    translations_ba = []
    rotations_ba_pred = []
    translations_ba_pred = []

    eulers_ab = []
    eulers_ba = []

    net = net.cuda()

    for src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba, _ in train_loader:
        # src = torch.tensor(src)
        # target = torch.tensor(target)
        src = src.cuda()
        target = target.cuda()
        rotation_ab = rotation_ab.cuda()
        translation_ab = translation_ab.cuda()
        rotation_ba = rotation_ba.cuda()
        translation_ba = translation_ba.cuda()

        batch_size = src.size(0)
        opt.zero_grad()
        num_examples += batch_size
        rotation_ab_pred, translation_ab_pred, rotation_ba_pred, translation_ba_pred = net(src, target)

        ## save rotation and translation
        rotations_ab.append(rotation_ab.detach().cpu().numpy())
        translations_ab.append(translation_ab.detach().cpu().numpy())
        rotations_ab_pred.append(rotation_ab_pred.detach().cpu().numpy())
        translations_ab_pred.append(translation_ab_pred.detach().cpu().numpy())
        eulers_ab.append(euler_ab.numpy())
        ##
        rotations_ba.append(rotation_ba.detach().cpu().numpy())
        translations_ba.append(translation_ba.detach().cpu().numpy())
        rotations_ba_pred.append(rotation_ba_pred.detach().cpu().numpy())
        translations_ba_pred.append(translation_ba_pred.detach().cpu().numpy())
        eulers_ba.append(euler_ba.numpy())

        transformed_src = transform_point_cloud(src, rotation_ab_pred, translation_ab_pred)

        transformed_target = transform_point_cloud(target, rotation_ba_pred, translation_ba_pred)
        ###########################
        identity = torch.eye(3).cuda().unsqueeze(0).repeat(batch_size, 1, 1)
        loss = F.mse_loss(torch.matmul(rotation_ab_pred.transpose(2, 1), rotation_ab), identity) \
               + F.mse_loss(translation_ab_pred, translation_ab)
        if args['cycle']:
            rotation_loss = F.mse_loss(torch.matmul(rotation_ba_pred, rotation_ab_pred), identity.clone())
            translation_loss = torch.mean((torch.matmul(rotation_ba_pred.transpose(2, 1),
                                                        translation_ab_pred.view(batch_size, 3, 1)).view(batch_size, 3)
                                           + translation_ba_pred) ** 2, dim=[0, 1])
            cycle_loss = rotation_loss + translation_loss

            loss = loss + cycle_loss * 0.1

        loss.backward()
        opt.step()
        total_loss += loss.item() * batch_size

        if args['cycle']:
            total_cycle_loss = total_cycle_loss + cycle_loss.item() * 0.1 * batch_size

        mse_ab += torch.mean((transformed_src - target) ** 2, dim=[0, 1, 2]).item() * batch_size
        mae_ab += torch.mean(torch.abs(transformed_src - target), dim=[0, 1, 2]).item() * batch_size

        mse_ba += torch.mean((transformed_target - src) ** 2, dim=[0, 1, 2]).item() * batch_size
        mae_ba += torch.mean(torch.abs(transformed_target - src), dim=[0, 1, 2]).item() * batch_size

    rotations_ab = np.concatenate(rotations_ab, axis=0)
    translations_ab = np.concatenate(translations_ab, axis=0)
    rotations_ab_pred = np.concatenate(rotations_ab_pred, axis=0)
    translations_ab_pred = np.concatenate(translations_ab_pred, axis=0)

    rotations_ba = np.concatenate(rotations_ba, axis=0)
    translations_ba = np.concatenate(translations_ba, axis=0)
    rotations_ba_pred = np.concatenate(rotations_ba_pred, axis=0)
    translations_ba_pred = np.concatenate(translations_ba_pred, axis=0)

    eulers_ab = np.concatenate(eulers_ab, axis=0)
    eulers_ba = np.concatenate(eulers_ba, axis=0)

    return total_loss * 1.0 / num_examples, total_cycle_loss / num_examples, \
           mse_ab * 1.0 / num_examples, mae_ab * 1.0 / num_examples, \
           mse_ba * 1.0 / num_examples, mae_ba * 1.0 / num_examples, rotations_ab, \
           translations_ab, rotations_ab_pred, translations_ab_pred, rotations_ba, \
           translations_ba, rotations_ba_pred, translations_ba_pred, eulers_ab, eulers_ba

def train(args, net, train_loader, test_loader, boardio, textio):
    if args['use_sgd']:
        print("Use SGD")
        opt = optim.SGD(net.parameters(), lr=args['lr'] * 100, momentum=args.momentum, weight_decay=1e-4)
    else:
        print("Use Adam")
        opt = optim.Adam(net.parameters(), lr=args['lr'], weight_decay=1e-4)
    scheduler = MultiStepLR(opt, milestones=[75, 150, 200], gamma=0.1)


    best_test_loss = np.inf
    best_test_cycle_loss = np.inf
    best_test_mse_ab = np.inf
    best_test_rmse_ab = np.inf
    best_test_mae_ab = np.inf

    best_test_r_mse_ab = np.inf
    best_test_r_rmse_ab = np.inf
    best_test_r_mae_ab = np.inf
    best_test_t_mse_ab = np.inf
    best_test_t_rmse_ab = np.inf
    best_test_t_mae_ab = np.inf

    best_test_mse_ba = np.inf
    best_test_rmse_ba = np.inf
    best_test_mae_ba = np.inf

    best_test_r_mse_ba = np.inf
    best_test_r_rmse_ba = np.inf
    best_test_r_mae_ba = np.inf
    best_test_t_mse_ba = np.inf
    best_test_t_rmse_ba = np.inf
    best_test_t_mae_ba = np.inf

    for epoch in tqdm(range(args['epochs'])):
        scheduler.step()
        train_loss, train_cycle_loss, \
        train_mse_ab, train_mae_ab, train_mse_ba, train_mae_ba, train_rotations_ab, train_translations_ab, \
        train_rotations_ab_pred, \
        train_translations_ab_pred, train_rotations_ba, train_translations_ba, train_rotations_ba_pred, \
        train_translations_ba_pred, train_eulers_ab, train_eulers_ba = train_one_epoch(args, net, train_loader, opt)
        test_loss, test_cycle_loss, \
        test_mse_ab, test_mae_ab, test_mse_ba, test_mae_ba, test_rotations_ab, test_translations_ab, \
        test_rotations_ab_pred, \
        test_translations_ab_pred, test_rotations_ba, test_translations_ba, test_rotations_ba_pred, \
        test_translations_ba_pred, test_eulers_ab, test_eulers_ba = test_one_epoch(args, net, test_loader)
        train_rmse_ab = np.sqrt(train_mse_ab)
        test_rmse_ab = np.sqrt(test_mse_ab)

        train_rmse_ba = np.sqrt(train_mse_ba)
        test_rmse_ba = np.sqrt(test_mse_ba)

        train_rotations_ab_pred_euler = npmat2euler(train_rotations_ab_pred)
        train_r_mse_ab = np.mean((train_rotations_ab_pred_euler - np.degrees(train_eulers_ab)) ** 2)
        train_r_rmse_ab = np.sqrt(train_r_mse_ab)
        train_r_mae_ab = np.mean(np.abs(train_rotations_ab_pred_euler - np.degrees(train_eulers_ab)))
        train_t_mse_ab = np.mean((train_translations_ab - train_translations_ab_pred) ** 2)
        train_t_rmse_ab = np.sqrt(train_t_mse_ab)
        train_t_mae_ab = np.mean(np.abs(train_translations_ab - train_translations_ab_pred))

        train_rotations_ba_pred_euler = npmat2euler(train_rotations_ba_pred, 'xyz')
        train_r_mse_ba = np.mean((train_rotations_ba_pred_euler - np.degrees(train_eulers_ba)) ** 2)
        train_r_rmse_ba = np.sqrt(train_r_mse_ba)
        train_r_mae_ba = np.mean(np.abs(train_rotations_ba_pred_euler - np.degrees(train_eulers_ba)))
        train_t_mse_ba = np.mean((train_translations_ba - train_translations_ba_pred) ** 2)
        train_t_rmse_ba = np.sqrt(train_t_mse_ba)
        train_t_mae_ba = np.mean(np.abs(train_translations_ba - train_translations_ba_pred))

        test_rotations_ab_pred_euler = npmat2euler(test_rotations_ab_pred)
        test_r_mse_ab = np.mean((test_rotations_ab_pred_euler - np.degrees(test_eulers_ab)) ** 2)
        test_r_rmse_ab = np.sqrt(test_r_mse_ab)
        test_r_mae_ab = np.mean(np.abs(test_rotations_ab_pred_euler - np.degrees(test_eulers_ab)))
        test_t_mse_ab = np.mean((test_translations_ab - test_translations_ab_pred) ** 2)
        test_t_rmse_ab = np.sqrt(test_t_mse_ab)
        test_t_mae_ab = np.mean(np.abs(test_translations_ab - test_translations_ab_pred))

        test_rotations_ba_pred_euler = npmat2euler(test_rotations_ba_pred, 'xyz')
        test_r_mse_ba = np.mean((test_rotations_ba_pred_euler - np.degrees(test_eulers_ba)) ** 2)
        test_r_rmse_ba = np.sqrt(test_r_mse_ba)
        test_r_mae_ba = np.mean(np.abs(test_rotations_ba_pred_euler - np.degrees(test_eulers_ba)))
        test_t_mse_ba = np.mean((test_translations_ba - test_translations_ba_pred) ** 2)
        test_t_rmse_ba = np.sqrt(test_t_mse_ba)
        test_t_mae_ba = np.mean(np.abs(test_translations_ba - test_translations_ba_pred))

        if best_test_loss >= test_loss:
            best_test_loss = test_loss
            best_test_cycle_loss = test_cycle_loss

            best_test_mse_ab = test_mse_ab
            best_test_rmse_ab = test_rmse_ab
            best_test_mae_ab = test_mae_ab

            best_test_r_mse_ab = test_r_mse_ab
            best_test_r_rmse_ab = test_r_rmse_ab
            best_test_r_mae_ab = test_r_mae_ab

            best_test_t_mse_ab = test_t_mse_ab
            best_test_t_rmse_ab = test_t_rmse_ab
            best_test_t_mae_ab = test_t_mae_ab

            best_test_mse_ba = test_mse_ba
            best_test_rmse_ba = test_rmse_ba
            best_test_mae_ba = test_mae_ba

            best_test_r_mse_ba = test_r_mse_ba
            best_test_r_rmse_ba = test_r_rmse_ba
            best_test_r_mae_ba = test_r_mae_ba

            best_test_t_mse_ba = test_t_mse_ba
            best_test_t_rmse_ba = test_t_rmse_ba
            best_test_t_mae_ba = test_t_mae_ba

            if torch.cuda.device_count() > 1:
                torch.save(net.module.state_dict(), 'checkpoints/%s/models/model.best.t7' % args['exp_name'])
            else:
                torch.save(net.state_dict(), 'checkpoints/%s/models/model.best.t7' % args['exp_name'])

        # textio.cprint('==TRAIN==')
        # textio.cprint('A--------->B')
        # textio.cprint('EPOCH:: %d, Loss: %f, Cycle Loss:, %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
        #               'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f'
        #               % (epoch, train_loss, train_cycle_loss, train_mse_ab, train_rmse_ab, train_mae_ab, train_r_mse_ab,
        #                  train_r_rmse_ab, train_r_mae_ab, train_t_mse_ab, train_t_rmse_ab, train_t_mae_ab))
        # textio.cprint('B--------->A')
        # textio.cprint('EPOCH:: %d, Loss: %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
        #               'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f'
        #               % (epoch, train_loss, train_mse_ba, train_rmse_ba, train_mae_ba, train_r_mse_ba, train_r_rmse_ba,
        #                  train_r_mae_ba, train_t_mse_ba, train_t_rmse_ba, train_t_mae_ba))

        # textio.cprint('==TEST==')
        # textio.cprint('A--------->B')
        # textio.cprint('EPOCH:: %d, Loss: %f, Cycle Loss: %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
        #               'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f'
        #               % (epoch, test_loss, test_cycle_loss, test_mse_ab, test_rmse_ab, test_mae_ab, test_r_mse_ab,
        #                  test_r_rmse_ab, test_r_mae_ab, test_t_mse_ab, test_t_rmse_ab, test_t_mae_ab))
        # textio.cprint('B--------->A')
        # textio.cprint('EPOCH:: %d, Loss: %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
        #               'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f'
        #               % (epoch, test_loss, test_mse_ba, test_rmse_ba, test_mae_ba, test_r_mse_ba, test_r_rmse_ba,
        #                  test_r_mae_ba, test_t_mse_ba, test_t_rmse_ba, test_t_mae_ba))

        # textio.cprint('==BEST TEST==')
        # textio.cprint('A--------->B')
        # textio.cprint('EPOCH:: %d, Loss: %f, Cycle Loss: %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
        #               'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f'
        #               % (epoch, best_test_loss, best_test_cycle_loss, best_test_mse_ab, best_test_rmse_ab,
        #                  best_test_mae_ab, best_test_r_mse_ab, best_test_r_rmse_ab,
        #                  best_test_r_mae_ab, best_test_t_mse_ab, best_test_t_rmse_ab, best_test_t_mae_ab))
        # textio.cprint('B--------->A')
        # textio.cprint('EPOCH:: %d, Loss: %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
        #               'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f'
        #               % (epoch, best_test_loss, best_test_mse_ba, best_test_rmse_ba, best_test_mae_ba,
        #                  best_test_r_mse_ba, best_test_r_rmse_ba,
        #                  best_test_r_mae_ba, best_test_t_mse_ba, best_test_t_rmse_ba, best_test_t_mae_ba))

        # boardio.add_scalar('A->B/train/loss', train_loss, epoch)
        # boardio.add_scalar('A->B/train/MSE', train_mse_ab, epoch)
        # boardio.add_scalar('A->B/train/RMSE', train_rmse_ab, epoch)
        # boardio.add_scalar('A->B/train/MAE', train_mae_ab, epoch)
        # boardio.add_scalar('A->B/train/rotation/MSE', train_r_mse_ab, epoch)
        # boardio.add_scalar('A->B/train/rotation/RMSE', train_r_rmse_ab, epoch)
        # boardio.add_scalar('A->B/train/rotation/MAE', train_r_mae_ab, epoch)
        # boardio.add_scalar('A->B/train/translation/MSE', train_t_mse_ab, epoch)
        # boardio.add_scalar('A->B/train/translation/RMSE', train_t_rmse_ab, epoch)
        # boardio.add_scalar('A->B/train/translation/MAE', train_t_mae_ab, epoch)

        # boardio.add_scalar('B->A/train/loss', train_loss, epoch)
        # boardio.add_scalar('B->A/train/MSE', train_mse_ba, epoch)
        # boardio.add_scalar('B->A/train/RMSE', train_rmse_ba, epoch)
        # boardio.add_scalar('B->A/train/MAE', train_mae_ba, epoch)
        # boardio.add_scalar('B->A/train/rotation/MSE', train_r_mse_ba, epoch)
        # boardio.add_scalar('B->A/train/rotation/RMSE', train_r_rmse_ba, epoch)
        # boardio.add_scalar('B->A/train/rotation/MAE', train_r_mae_ba, epoch)
        # boardio.add_scalar('B->A/train/translation/MSE', train_t_mse_ba, epoch)
        # boardio.add_scalar('B->A/train/translation/RMSE', train_t_rmse_ba, epoch)
        # boardio.add_scalar('B->A/train/translation/MAE', train_t_mae_ba, epoch)

        # ############TEST
        # boardio.add_scalar('A->B/test/loss', test_loss, epoch)
        # boardio.add_scalar('A->B/test/MSE', test_mse_ab, epoch)
        # boardio.add_scalar('A->B/test/RMSE', test_rmse_ab, epoch)
        # boardio.add_scalar('A->B/test/MAE', test_mae_ab, epoch)
        # boardio.add_scalar('A->B/test/rotation/MSE', test_r_mse_ab, epoch)
        # boardio.add_scalar('A->B/test/rotation/RMSE', test_r_rmse_ab, epoch)
        # boardio.add_scalar('A->B/test/rotation/MAE', test_r_mae_ab, epoch)
        # boardio.add_scalar('A->B/test/translation/MSE', test_t_mse_ab, epoch)
        # boardio.add_scalar('A->B/test/translation/RMSE', test_t_rmse_ab, epoch)
        # boardio.add_scalar('A->B/test/translation/MAE', test_t_mae_ab, epoch)

        # boardio.add_scalar('B->A/test/loss', test_loss, epoch)
        # boardio.add_scalar('B->A/test/MSE', test_mse_ba, epoch)
        # boardio.add_scalar('B->A/test/RMSE', test_rmse_ba, epoch)
        # boardio.add_scalar('B->A/test/MAE', test_mae_ba, epoch)
        # boardio.add_scalar('B->A/test/rotation/MSE', test_r_mse_ba, epoch)
        # boardio.add_scalar('B->A/test/rotation/RMSE', test_r_rmse_ba, epoch)
        # boardio.add_scalar('B->A/test/rotation/MAE', test_r_mae_ba, epoch)
        # boardio.add_scalar('B->A/test/translation/MSE', test_t_mse_ba, epoch)
        # boardio.add_scalar('B->A/test/translation/RMSE', test_t_rmse_ba, epoch)
        # boardio.add_scalar('B->A/test/translation/MAE', test_t_mae_ba, epoch)

        # ############BEST TEST
        # boardio.add_scalar('A->B/best_test/loss', best_test_loss, epoch)
        # boardio.add_scalar('A->B/best_test/MSE', best_test_mse_ab, epoch)
        # boardio.add_scalar('A->B/best_test/RMSE', best_test_rmse_ab, epoch)
        # boardio.add_scalar('A->B/best_test/MAE', best_test_mae_ab, epoch)
        # boardio.add_scalar('A->B/best_test/rotation/MSE', best_test_r_mse_ab, epoch)
        # boardio.add_scalar('A->B/best_test/rotation/RMSE', best_test_r_rmse_ab, epoch)
        # boardio.add_scalar('A->B/best_test/rotation/MAE', best_test_r_mae_ab, epoch)
        # boardio.add_scalar('A->B/best_test/translation/MSE', best_test_t_mse_ab, epoch)
        # boardio.add_scalar('A->B/best_test/translation/RMSE', best_test_t_rmse_ab, epoch)
        # boardio.add_scalar('A->B/best_test/translation/MAE', best_test_t_mae_ab, epoch)

        # boardio.add_scalar('B->A/best_test/loss', best_test_loss, epoch)
        # boardio.add_scalar('B->A/best_test/MSE', best_test_mse_ba, epoch)
        # boardio.add_scalar('B->A/best_test/RMSE', best_test_rmse_ba, epoch)
        # boardio.add_scalar('B->A/best_test/MAE', best_test_mae_ba, epoch)
        # boardio.add_scalar('B->A/best_test/rotation/MSE', best_test_r_mse_ba, epoch)
        # boardio.add_scalar('B->A/best_test/rotation/RMSE', best_test_r_rmse_ba, epoch)
        # boardio.add_scalar('B->A/best_test/rotation/MAE', best_test_r_mae_ba, epoch)
        # boardio.add_scalar('B->A/best_test/translation/MSE', best_test_t_mse_ba, epoch)
        # boardio.add_scalar('B->A/best_test/translation/RMSE', best_test_t_rmse_ba, epoch)
        # boardio.add_scalar('B->A/best_test/translation/MAE', best_test_t_mae_ba, epoch)

        if torch.cuda.device_count() > 1:
            torch.save(net.module.state_dict(), 'checkpoints/%s/models/model.%d.t7' % (args['exp_name'], epoch))
        else:
            torch.save(net.state_dict(), 'checkpoints/%s/models/model.%d.t7' % (args['exp_name'], epoch))
        gc.collect()

## Test

In [8]:
def test_one_epoch(args, net, test_loader):
    net.eval()
    mse_ab = 0
    mae_ab = 0
    mse_ba = 0
    mae_ba = 0

    total_loss = 0
    total_cycle_loss = 0
    num_examples = 0
    rotations_ab = []
    translations_ab = []
    rotations_ab_pred = []
    translations_ab_pred = []

    rotations_ba = []
    translations_ba = []
    rotations_ba_pred = []
    translations_ba_pred = []

    eulers_ab = []
    eulers_ba = []

    net = net.cuda()

    for src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba, _ in test_loader:
        src = src.cuda()
        target = target.cuda()
        rotation_ab = rotation_ab.cuda()
        translation_ab = translation_ab.cuda()
        rotation_ba = rotation_ba.cuda()
        translation_ba = translation_ba.cuda()
        # print(rotation_ab.shape)

        batch_size = src.size(0)
        num_examples += batch_size
        
        with torch.no_grad():
            rotation_ab_pred, translation_ab_pred, rotation_ba_pred, translation_ba_pred = net(src, target)

        ## save rotation and translation
        rotations_ab.append(rotation_ab.detach().cpu().numpy())
        translations_ab.append(translation_ab.detach().cpu().numpy())
        rotations_ab_pred.append(rotation_ab_pred.detach().cpu().numpy())
        translations_ab_pred.append(translation_ab_pred.detach().cpu().numpy())
        eulers_ab.append(euler_ab.numpy())
        ##
        rotations_ba.append(rotation_ba.detach().cpu().numpy())
        translations_ba.append(translation_ba.detach().cpu().numpy())
        rotations_ba_pred.append(rotation_ba_pred.detach().cpu().numpy())
        translations_ba_pred.append(translation_ba_pred.detach().cpu().numpy())
        eulers_ba.append(euler_ba.numpy())

        transformed_src = transform_point_cloud(src, rotation_ab_pred, translation_ab_pred)

        transformed_target = transform_point_cloud(target, rotation_ba_pred, translation_ba_pred)

        ###########################
        identity = torch.eye(3).cuda().unsqueeze(0).repeat(batch_size, 1, 1)
        loss = F.mse_loss(torch.matmul(rotation_ab_pred.transpose(2, 1), rotation_ab), identity) \
               + F.mse_loss(translation_ab_pred, translation_ab)

        total_loss += loss.item()
        mse_ab += torch.mean((transformed_src - target) ** 2, dim=[0, 1, 2]).item() * batch_size
        mae_ab += torch.mean(torch.abs(transformed_src - target), dim=[0, 1, 2]).item() * batch_size

        mse_ba += torch.mean((transformed_target - src) ** 2, dim=[0, 1, 2]).item() * batch_size
        mae_ba += torch.mean(torch.abs(transformed_target - src), dim=[0, 1, 2]).item() * batch_size

    rotations_ab = np.concatenate(rotations_ab, axis=0)
    translations_ab = np.concatenate(translations_ab, axis=0)
    rotations_ab_pred = np.concatenate(rotations_ab_pred, axis=0)
    translations_ab_pred = np.concatenate(translations_ab_pred, axis=0)

    rotations_ba = np.concatenate(rotations_ba, axis=0)
    translations_ba = np.concatenate(translations_ba, axis=0)
    rotations_ba_pred = np.concatenate(rotations_ba_pred, axis=0)
    translations_ba_pred = np.concatenate(translations_ba_pred, axis=0)

    eulers_ab = np.concatenate(eulers_ab, axis=0)
    eulers_ba = np.concatenate(eulers_ba, axis=0)

    return total_loss * 1.0 / num_examples, total_cycle_loss / num_examples, \
           mse_ab * 1.0 / num_examples, mae_ab * 1.0 / num_examples, \
           mse_ba * 1.0 / num_examples, mae_ba * 1.0 / num_examples, rotations_ab, \
           translations_ab, rotations_ab_pred, translations_ab_pred, rotations_ba, \
           translations_ba, rotations_ba_pred, translations_ba_pred, eulers_ab, eulers_ba


def test(args, net, test_loader, boardio, textio):

    test_loss, test_cycle_loss, \
    test_mse_ab, test_mae_ab, test_mse_ba, test_mae_ba, test_rotations_ab, test_translations_ab, \
    test_rotations_ab_pred, \
    test_translations_ab_pred, test_rotations_ba, test_translations_ba, test_rotations_ba_pred, \
    test_translations_ba_pred, test_eulers_ab, test_eulers_ba = test_one_epoch(args, net, test_loader)
    test_rmse_ab = np.sqrt(test_mse_ab)
    test_rmse_ba = np.sqrt(test_mse_ba)

    test_rotations_ab_pred_euler = npmat2euler(test_rotations_ab_pred)
    test_r_mse_ab = np.mean((test_rotations_ab_pred_euler - np.degrees(test_eulers_ab)) ** 2)
    test_r_rmse_ab = np.sqrt(test_r_mse_ab)
    test_r_mae_ab = np.mean(np.abs(test_rotations_ab_pred_euler - np.degrees(test_eulers_ab)))
    test_t_mse_ab = np.mean((test_translations_ab - test_translations_ab_pred) ** 2)
    test_t_rmse_ab = np.sqrt(test_t_mse_ab)
    test_t_mae_ab = np.mean(np.abs(test_translations_ab - test_translations_ab_pred))

    test_rotations_ba_pred_euler = npmat2euler(test_rotations_ba_pred, 'xyz')
    test_r_mse_ba = np.mean((test_rotations_ba_pred_euler - np.degrees(test_eulers_ba)) ** 2)
    test_r_rmse_ba = np.sqrt(test_r_mse_ba)
    test_r_mae_ba = np.mean(np.abs(test_rotations_ba_pred_euler - np.degrees(test_eulers_ba)))
    test_t_mse_ba = np.mean((test_translations_ba - test_translations_ba_pred) ** 2)
    test_t_rmse_ba = np.sqrt(test_t_mse_ba)
    test_t_mae_ba = np.mean(np.abs(test_translations_ba - test_translations_ba_pred))

    textio.cprint('==FINAL TEST==')
    textio.cprint('A--------->B')
    textio.cprint('EPOCH:: %d, Loss: %f, Cycle Loss: %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
                  'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f'
                  % (-1, test_loss, test_cycle_loss, test_mse_ab, test_rmse_ab, test_mae_ab,
                     test_r_mse_ab, test_r_rmse_ab,
                     test_r_mae_ab, test_t_mse_ab, test_t_rmse_ab, test_t_mae_ab))
    textio.cprint('B--------->A')
    textio.cprint('EPOCH:: %d, Loss: %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
                  'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f'
                  % (-1, test_loss, test_mse_ba, test_rmse_ba, test_mae_ba, test_r_mse_ba, test_r_rmse_ba,
                     test_r_mae_ba, test_t_mse_ba, test_t_rmse_ba, test_t_mae_ba))

    return test_rotations_ab_pred, test_translations_ab_pred

# DCP

In [34]:
import open3d as o3d
import numpy as np
from torch.utils.data import Dataset
class TUM(Dataset):
        def __init__(self, size):
                self.pcd1 = o3d.io.read_point_cloud("scenenet/val/0/0/3800_after_filtering.pcd")
                self.index = np.random.randint(len(self.pcd1.points),size=size)
                # print(self.index)
                self.points1 = np.array(self.pcd1.points)[self.index] #转为矩阵

                self.pcd2 = o3d.io.read_point_cloud("scenenet/val/0/0/3800_after_filtering.pcd")
                self.points2 = np.array(self.pcd2.points)[self.index]

                self.points1 = self.points1[np.newaxis,:]
    
        def __getitem__(self, index):
                points1 = self.points1[index]
                anglex = np.random.uniform() * np.pi / 20
                angley = np.random.uniform() * np.pi / 20
                anglez = np.random.uniform() * np.pi / 20

                anglex = 0.10750457315556647
                angley = -0.16308165329963306
                anglez = 0.011883495138386518

                cosx = np.cos(anglex)
                cosy = np.cos(angley)
                cosz = np.cos(anglez)
                sinx = np.sin(anglex)
                siny = np.sin(angley)
                sinz = np.sin(anglez)
                Rx = np.array([[1, 0, 0],
                                [0, cosx, -sinx],
                                [0, sinx, cosx]])
                Ry = np.array([[cosy, 0, siny],
                                [0, 1, 0],
                                [-siny, 0, cosy]])
                Rz = np.array([[cosz, -sinz, 0],
                                [sinz, cosz, 0],
                                [0, 0, 1]])
                R_ab = Rx.dot(Ry).dot(Rz)
                R_ba = R_ab.T
                translation_ab = np.array([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5),
                                                np.random.uniform(-0.5, 0.5)])
                tx = 1.0953829982311267
                ty = 0.4325993321892744
                tz = 0.4725850417356405
                translation_ab = np.array([tx, ty, tz])
                translation_ba = -R_ba.dot(translation_ab)


                euler_ab = np.asarray([anglez, angley, anglex])
                euler_ba = -euler_ab[::-1]
                
                return points1.T.astype('float32'), self.points2.T.astype('float32'), R_ab.astype('float32'), \
        translation_ab.astype('float32'), R_ba.astype('float32'), translation_ba.astype('float32'), \
        euler_ab.astype('float32'), euler_ba.astype('float32'), self.index

        def __len__(self):
                return len(self.points1)

points = TUM(20000)

# define the dataloader
# pc = ModelNet40(num_points=1024, partition='test', gaussian_noise=False, unseen=False, factor=4)
# src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba = pc[6]

args = {
        "model_path": 'checkpoints/dcp_v1/models/model.best.t7',
        # "model_path": 'dcp-master/pretrained/dcp_v1.t7',
        "exp_name":"dcp_v1",
         "model":"dcp", 
         "emb_nn":"dgcnn", 
         "pointer":"identity", 
         "head":"svd", 
         "eval": True,
         'emb_dims': 512,
         'cycle': False,
         'use_sgd': False,
         'lr': 0.001,
         'epochs': 250,
         'n_blocks': 1,
         'dropout': 0.0,
         'ff_dims': 1024,
         'n_heads': 4,
        }

train_loader = DataLoader(points, batch_size=32, shuffle=True, drop_last=False)
test_loader = DataLoader(points, batch_size=1, shuffle=False, drop_last=False)

# test_loader = DataLoader(
#             ModelNet40(num_points=args.num_points, partition='test', gaussian_noise=args.gaussian_noise,
#                        unseen=args.unseen, factor=args.factor),
#             batch_size=args.test_batch_size, shuffle=False, drop_last=False)
boardio = SummaryWriter(log_dir='checkpoints/' + args['exp_name'])
textio = IOStream('checkpoints/' + args['exp_name'] + '/run.log')
textio.cprint(str(args))
net = DCP(args)

net.train()
train(args, net, train_loader, test_loader, boardio, textio)

net.load_state_dict(torch.load(args['model_path']), strict=False)
net.eval()
r, t = test(args, net, test_loader, boardio, textio)

print(r)
print(t)

[Open3D WARNING] [CheckHeader] PCD has no data.
[Open3D WARNING] Read PCD failed: unable to parse header.


ValueError: high <= 0

# Visualisation

In [6]:
import matplotlib.pyplot as plt
import k3d

def visualize_pointcloud(point_cloud1, point_size, point_cloud2=None, flip_axes=False, name='point_cloud', R=None, t=None):
    plot = k3d.plot(name=name, grid_visible=False, grid=(-0.55, -0.55, -0.55, 0.55, 0.55, 0.55))
    # if flip_axes:
    #     point_cloud[:, 2] = point_cloud[:, 2] * -1
    #     point_cloud[:, [0, 1, 2]] = point_cloud[:, [0, 2, 1]]
    # t_broadcast = np.broadcast_to(t[:, np.newaxis], (3, point_cloud1.X.shape[0]))
    # point_cloud1 = (R @ point_cloud1.X.T + t_broadcast).T
    plt_points1 = k3d.points(positions=point_cloud1, point_size=point_size, color=0xd0d0d0)
    plot += plt_points1
    plt_points1.shader = '3d'
    if point_cloud2 is not None:
        plt_points2 = k3d.points(positions=point_cloud2, point_size=point_size, color=0x0dd00d)
        plot += plt_points2
        plt_points2.shader = '3d'
    plot.display()

def transform(point_cloud, R=None, t=None):
    t_broadcast = np.broadcast_to(t[:, np.newaxis], (3, point_cloud.shape[0]))
    return (R @ point_cloud.T + t_broadcast).T


src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba, index = points[0]
print(src.shape)
quat = torch.tensor([[0.6574, 0.6126, -0.2949, -0.3248], [0.6579, 0.6161, -0.2932, -0.3189]])
r = torch.tensor([[[ 0.99959856, -0.00936801,  0.02673812],
 [ 0.01243275,  0.9930703,  -0.1168622 ],
 [-0.02545807,  0.11714771,  0.99278814]]])
# print(r)
t = torch.tensor([[-0.29480037,  0.11526811,  0.31008014]])

r1 = torch.tensor([[[ 0.98670018, -0.01175757, -0.162125],
 [-0.00559273,  0.99433457, -0.10614836],
 [ 0.16245453,  0.10564333,  0.98104435]]])
t1 = torch.tensor([[1.09464561,  0.43318735,   0.47305842 ]])
# print(points1.shape)
transformed_src = transform_point_cloud(torch.tensor(src), r, t).T
transformed_src1 = transform_point_cloud(torch.tensor(src), r1, t1).T
visualize_pointcloud(target.T, .03, transformed_src1)
visualize_pointcloud(target.T, .03, transformed_src)
visualize_pointcloud(src.T, .03, target.T)


(3, 20000)


/var/folders/cz/1xlnx7y504g5_dyp82wtzxhc0000gn/T/ipykernel_68646/3921062681.py:39: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/TensorShape.cpp:2318.)
  transformed_src = transform_point_cloud(torch.tensor(src), r, t).T


Output()

Output()

Output()

In [37]:
r = np.array([[ 0.00520319,  0.62180531, -0.00795137],
 [ 0.61245382,  0.00143963, -0.04598352],
 [-0.0056224,  -0.05133275,  0.3842194 ]])

# r1 = np.array([[ 0.07543147,  0.61393189, -0.78574661],
#  [ 0.9970987,  -0.03837025,  0.06574118],
#  [ 0.01021131, -0.78842588, -0.61504501]])

r2 = np.array([[ 0.06897899,  0.62361461, -0.77868268],
 [ 0.99759245, -0.03751941,  0.0583232 ],
 [ 0.00715548, -0.78083104, -0.62470127]])

r = np.array([[ 1,  -0.02185709,  0.08722663],
  [ 0.08445041,  1,   2.25380598],
  [-0.02161435, 1.43743747,  1 ]])
print(np.linalg.det(r1))
print(np.dot(r2, r1.T))
print(npmat2euler(r1)+2*np.pi)
rx = 0.10750457315556647
ry = -0.16308165329963306
rz = 0.011883495138386518
tx = 1.0953829982311267
ty = 0.4325993321892744
tz = 0.4725850417356405
def euler_angles_to_rotation_matrix(
    alpha1: float, alpha2: float, alpha3: float
) -> np.array:
    """Get Euler angles from rotation matrix."""

    R = np.array(
        [
            [
                np.cos(alpha2) * np.cos(alpha3),
                -np.cos(alpha2) * np.sin(alpha3),
                np.sin(alpha2),
            ],
            [
                np.cos(alpha1) * np.sin(alpha3)
                + np.sin(alpha1) * np.sin(alpha2) * np.cos(alpha3),
                np.cos(alpha1) * np.cos(alpha3)
                - np.sin(alpha1) * np.sin(alpha2) * np.sin(alpha3),
                -np.sin(alpha1) * np.cos(alpha2),
            ],
            [
                np.sin(alpha1) * np.sin(alpha3)
                - np.cos(alpha1) * np.sin(alpha2) * np.cos(alpha3),
                np.sin(alpha1) * np.cos(alpha3)
                + np.cos(alpha1) * np.sin(alpha2) * np.sin(alpha3),
                np.cos(alpha1) * np.cos(alpha2),
            ],
        ]
    )

    return R
print(euler_angles_to_rotation_matrix(rx, ry, rz))
print(np.array([tx, ty, tz]))

[1.]
[[[-0.06211722]
  [ 0.53723976]
  [-0.84113905]]

 [[ 0.9939852 ]
  [-0.04285963]
  [-0.10077942]]

 [[-0.09019362]
  [-0.84233992]
  [-0.53134601]]]
[[ 6.9640594 -3.0607047 12.442743 ]]
[[ 0.98666196 -0.01172554 -0.16235974]
 [-0.00560497  0.99436376 -0.10587395]
 [ 0.16268607  0.10537182  0.98103518]]
[1.095383   0.43259933 0.47258504]


In [12]:
viewpose1 = {
    'lookat': np.array([4.7825798988342285, 1.166599988937378, 4.573699951171875]),
    'camera': np.array([-1.0683399438858032, 0.7407330274581909, 4.073339939117432])
}

viewpose2 = {
    'lookat': np.array([4.7150797843933105, 0.9783260226249695, 4.412020206451416]),
    'camera': np.array([-1.2428200244903564, 1.1899399757385254, 2.8996899127960205])
}

def normalize(v):
    return v/np.linalg.norm(v)


def world_to_camera_with_pose(view_pose):
    lookat_pose = view_pose['lookat']
    camera_pose = view_pose['camera']
    up = np.array([0,1,0])
    R = np.diag(np.ones(4))
    R[2,:3] = normalize(lookat_pose - camera_pose)
    R[0,:3] = normalize(np.cross(R[2,:3],up))
    R[1,:3] = -normalize(np.cross(R[0,:3],R[2,:3]))
    T = np.diag(np.ones(4))
    T[:3,3] = -camera_pose
    return R.dot(T)


def camera_to_world_with_pose(view_pose):
    return np.linalg.inv(world_to_camera_with_pose(view_pose))

r1 = world_to_camera_with_pose(viewpose1)
r2 = world_to_camera_with_pose(viewpose2)

print(r1)
print(r2)

print(np.dot(r2, np.linalg.inv(r1)))


[[-0.08520717  0.          0.99636326 -4.14955647]
 [ 0.07206861 -0.99738063  0.00616318  0.79068184]
 [ 0.99375342  0.07233166  0.08498398  0.66191937]
 [ 0.          0.          0.          1.        ]]
[[-0.24603355  0.          0.96926131 -3.11633267]
 [-0.03334845 -0.99940794 -0.00846504  1.17233533]
 [ 0.96868745 -0.03440605  0.24588788  0.53184667]
 [ 0.          0.          0.          1.        ]]
[[ 0.98670018 -0.01175757 -0.162125    1.09464561]
 [-0.00559273  0.99433457 -0.10614836  0.43318735]
 [ 0.16245453  0.10564333  0.98104435  0.47305842]
 [ 0.          0.          0.          1.        ]]
